In [9]:
import numpy as np
import matplotlib.pyplot as plt

def smooth_cepheid_lightcurve(phase, amplitude=0.5, skew=0.3):
    """
    Generates a smooth asymmetric light curve.
    - phase: values between 0 and 1
    - skew: between 0 and 1; smaller = faster rise, larger = slower rise
    """
    brightness = np.zeros_like(phase)
    # Rising phase (steep)
    rise = phase < skew
    brightness[rise] = amplitude * (1 - np.cos(np.pi * phase[rise] / skew)) / 2
    # Falling phase (shallow)
    fall = ~rise
    brightness[fall] = amplitude * (1 + np.cos(np.pi * (phase[fall] - skew) / (1 - skew))) / 2
    return -brightness  # Negative because lower magnitude = brighter

# Define cepheid properties
cepheids = {
    "A": {"period": 5.4, "mag": 15.2, "color": "blue"},
    "B": {"period": 10.1, "mag": 14.1, "color": "green"},
    "C": {"period": 3.3, "mag": 16.0, "color": "red"},
}

time = np.linspace(0, 20, 1000)
plt.figure(figsize=(8, 5))

for name, props in cepheids.items():
    period = props["period"]
    mag = props["mag"]
    amplitude = 0.4
    skew = 0.3
    
    phase = (time % period) / period
    lc = mag + smooth_cepheid_lightcurve(phase, amplitude=amplitude, skew=skew)
    plt.plot(time, lc, label=f"Cepheid {name}", color=props["color"])

plt.gca().invert_yaxis()
plt.xlabel("Time (days)")
plt.ylabel("Apparent Magnitude")
plt.title("Smoothed Asymmetric Cepheid Light Curves")
plt.legend()
plt.tight_layout()
plt.savefig("cephid_lightcurves.png", dpi=300)
plt.close()


In [10]:
# --- DISTANCE CALCULATION & PLOT ---
def compute_absolute_magnitude(P):
    μ = -1.0
    return -2.76 * np.log10(P) + μ

def compute_distance(m, M):
    return 10 ** ((m - M + 5) / 5)

distances = []
names = []

for name, props in cepheids.items():
    P = props["period"]
    m = props["mag"]
    M = compute_absolute_magnitude(P)
    d = compute_distance(m, M)
    distances.append(d)
    names.append(f"Cepheid {name}")

# --- DISTANCE BAR CHART ---
plt.figure(figsize=(6, 4))
plt.bar(names, distances, color=["blue", "green", "red"])
plt.ylabel("Distance (parsecs)")
plt.title("Estimated Distances to Cepheid Stars")
plt.tight_layout()
plt.savefig("cephid_distances.png", dpi=300)
plt.close()
